# Vietnamese Healthcare Assistant

## Libraries

In [1]:
def install_kaggle():
  !mamba install --force-reinstall aiohttp -y
  !pip install -U "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
  !pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

  # Temporary fix for https://github.com/huggingface/datasets/issues/6753
  !pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

install_kaggle()


Looking for: ['aiohttp']

warning  libmamba Cache file "/opt/conda/pkgs/cache/c6f2354e.json" was modified by another program
[+] 0.0s
[+] 0.1s
rapidsai/linux-64 (check zst) ━━━━━━━━━━━━━━━   0.0 B @  ??.?MB/s Checking  0.1s[+] 0.2s
rapidsai/linux-64 (check zst) ━━━━━━━━━━━━━━━   0.0 B @  ??.?MB/s Checking  0.2s[+] 0.3s
rapidsai/linux-64 (check zst) ━━━━━━━━━━━━━━━   0.0 B @  ??.?MB/s Checking  0.3srapidsai/linux-64 (check zst)                       Checked  0.3s
warning  libmamba Cache file "/opt/conda/pkgs/cache/86b0f08d.json" was modified by another program
[+] 0.0s
[+] 0.1s
rapidsai/noarch (check zst) ━━━━━━━━━━━╸━━━━━   0.0 B @  ??.?MB/s Checking  0.1srapidsai/noarch (check zst)                         Checked  0.1s
warning  libmamba Cache file "/opt/conda/pkgs/cache/c9ddbd6b.json" was modified by another program
[+] 0.0s
nvidia/linux-64 (check zst)                        Checked  0.1s
warning  libmamba Cache file "/opt/conda/pkgs/cache/b121c3e7.json" was modified by another progr

In [2]:
from dataclasses import dataclass, fields
from typing import List, Optional

import torch
from unsloth import (
    FastLanguageModel,
    is_bfloat16_supported,
)
from unsloth.chat_templates import get_chat_template
from transformers import TrainingArguments
from trl import SFTTrainer
from datasets import load_dataset, Dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


## Load model & tokenizer

Here are 4-bit model supported by **Unsloth** in present:

```python
[
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
]
```

And in this notebook, we will use `unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit` model

In [18]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    load_in_4bit = True
)

==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: Tesla P100-PCIE-16GB. Max memory: 15.888 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 6.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.25.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

Add LoRA adapter

In [19]:
# All values passed are default (you can check by `help(FastLanguageModel.get_peft_model)`)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,                                                   # default value
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj', # default values
                      'gate_proj', 'up_proj', 'down_proj'],
    lora_dropout = 0,                                         # default value
    bias = 'none',                                            # default value
    use_gradient_checkpointing = True,                        # default value
    random_state = 3047,                                      # default value
    use_rslora = False,                                       # default value
    loftq_config = None,                                      # default value
)

Unsloth 2024.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Data Preparation

In this step, we will prepare data (include loading and loading to transform raw data into valid data for LLM)

The ultimate goal is creating an assistant to consult, resolve problems relating to medical and health for users. This assistant will be proficient in answering by both Vietnamese and English. However, at the beginning stage of this project, we will only fine-tune on **ViHealthQA** and **Identification** dataset.

**Dataset**

| Name | size | link | note |
|------------|------------|------------|------------|
| **tarudesu/ViHealthQA** | 7.01k | https://huggingface.co/datasets/tarudesu/ViHealthQA | |
| **BookingCare/ViHealthCorpus** | 37.4K | https://huggingface.co/datasets/BookingCare/ViHealthCorpus | need to convert to chat format |
| **Identification dataset** | | | aims at identifying assistant |


***Note:* A good fine-tuning process is following the prompt template of corresponding model (we will do it later)**

### Utils

In [11]:
def pair_to_shareGPT(
    row: dict,
    mapping: dict = {'user': 'question', 'assistant': 'answer'},
    target_col: str = 'conversation',
    system_prompt: Optional[str] = None
):
    res = []
    if system_prompt:
        res.append({'from': 'system', 'value': system_prompt}) 
    for k, v in mapping.items():
        res.append({'from': k, 'value': row[v]})
        
    return {target_col: res}

### Load

In [34]:
vihealth_dataset = load_dataset("tarudesu/ViHealthQA")

In [32]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    texts = []
    for e in examples:
        text = e['question'] + ' ' + e['answer'] + EOS_TOKEN
    return { "text" : texts, }

In [36]:
dataset = dataset.map(formatting_prompts_func)

Map:   0%|          | 0/7009 [00:00<?, ? examples/s]

TypeError: string indices must be integers

## Train

In [28]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = vihealth_dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

KeyError: "Invalid key: 0. Please first select a split. For example: `my_dataset_dictionary['train'][0]`. Available splits: ['test', 'train', 'validation']"